In [164]:
import requests
import json
import boto3

In [165]:

ACCESS_KEY = ''
SECRET_KEY = ''
API_KEY = ''


In [166]:
# S3
REGION = 'ca-central-1'
BUCKET = 'quetzal-immense'

# api gateway
url = 'https://lud5uqi5j5.execute-api.ca-central-1.amazonaws.com/dev'
stateMachineArn = 'arn:aws:states:ca-central-1:142023388927:stateMachine:quetzal-immense'

In [167]:
headers = {
        'Accept': '*/*',
        'Content-Type':'application/json',
        'x-api-key' : API_KEY,
}

# describe

In [168]:
data = { "stateMachineArn": stateMachineArn }
data=json.dumps(data)
data

'{"stateMachineArn": "arn:aws:states:ca-central-1:142023388927:stateMachine:quetzal-immense"}'

In [169]:
resp =  requests.post(url+'/describe/model/', data=data, headers=headers)
resp.ok

True

In [170]:
resp.json()

{'creationDate': 1707159295.38,
 'definition': '{\n  "StartAt": "Choice",\n  "States": {\n    "Choice": {\n      "Type": "Choice",\n      "Choices": [\n        {\n          "Variable": "$.choice",\n          "StringEquals": "demand",\n          "Next": "Demand"\n        },\n        {\n          "Variable": "$.choice",\n          "StringEquals": "orchestrator",\n          "Next": "Parallel"\n        }\n      ],\n      "Default": "Network Preparation"\n    },\n    "Network Preparation": {\n      "Type": "Task",\n      "Resource": "arn:aws:states:::lambda:invoke",\n      "OutputPath": "$.Payload",\n      "Parameters": {\n        "Payload": {\n          "notebook_path": "notebooks/2_model/B10_network_preparation.ipynb",\n          "scenario_path_S3.$": "$.scenario_path_S3",\n          "launcher_arg.$": "$.launcher_arg",\n          "metadata.$": "$.metadata"\n        },\n        "FunctionName": "arn:aws:lambda:ca-central-1:142023388927:function:quetzal-immense:$LATEST"\n      },\n      "Ret

# start model

All parameters passed to the function or in params

In [144]:
params = {"param1":"test","param2":3}
launcher_arg = {'training_folder': '/tmp','params': params}
inputs = { 
        'scenario_path_S3': 'test/',
        'choice':'default',
        'launcher_arg': launcher_arg,
        'metadata' : {"user_email": "test@test.com"}
}
inputs = json.dumps(inputs)

In [145]:
data = { "stateMachineArn": stateMachineArn, 'input':inputs }
data = json.dumps(data)
data


'{"stateMachineArn": "arn:aws:states:ca-central-1:142023388927:stateMachine:quetzal-immense", "input": "{\\"scenario_path_S3\\": \\"test/\\", \\"choice\\": \\"default\\", \\"launcher_arg\\": {\\"training_folder\\": \\"/tmp\\", \\"params\\": {\\"param1\\": \\"test\\", \\"param2\\": 3}}, \\"metadata\\": {\\"user_email\\": \\"test@test.com\\"}}"}'

In [146]:
resp =  requests.post(url,data=data,headers=headers)
resp.ok

True

In [147]:
resp.json()

{'executionArn': 'arn:aws:states:ca-central-1:142023388927:execution:quetzal-immense:6ebc2884-9ea1-4c3c-ae60-662d2668b6f4',
 'startDate': 1707344201.299}

In [148]:
# get executionARN for polling
resp.json()['executionArn']

'arn:aws:states:ca-central-1:142023388927:execution:quetzal-immense:6ebc2884-9ea1-4c3c-ae60-662d2668b6f4'

# get status (polling)

In [149]:
executionArn = resp.json()['executionArn']
data = { "executionArn": executionArn }
data=json.dumps(data, ensure_ascii=False).encode('utf-8')
resp =  requests.post(url+'/describe',data=data,headers=headers)
resp.ok

True

In [150]:
resp.json()

{'executionArn': 'arn:aws:states:ca-central-1:142023388927:execution:quetzal-immense:6ebc2884-9ea1-4c3c-ae60-662d2668b6f4',
 'input': '{"scenario_path_S3": "test/", "choice": "default", "launcher_arg": {"training_folder": "/tmp", "params": {"param1": "test", "param2": 3}}, "metadata": {"user_email": "test@test.com"}}',
 'inputDetails': {'__type': 'com.amazonaws.swf.base.model#CloudWatchEventsExecutionDataDetails',
  'included': True},
 'name': '6ebc2884-9ea1-4c3c-ae60-662d2668b6f4',
 'redriveCount': 0,
 'redriveStatus': 'NOT_REDRIVABLE',
 'redriveStatusReason': 'Execution is RUNNING and cannot be redriven',
 'startDate': 1707344201.299,
 'stateMachineArn': 'arn:aws:states:ca-central-1:142023388927:stateMachine:quetzal-immense',
 'status': 'RUNNING',
 'traceHeader': 'Root=1-65c40149-197befda2d91b3215bd9e39a;Sampled=1'}

In [151]:
resp.json()['status']

'RUNNING'

In [152]:
import time

status =  resp.json()['status']
while status == 'RUNNING':
    resp =  requests.post(url+'/describe',data=data,headers=headers)
    status = resp.json()['status']
    print(status)
    time.sleep(2)

RUNNING
RUNNING
SUCCEEDED


SUCCEEDED or ERROR

# stop

In [30]:
executionArn

'arn:aws:states:ca-central-1:142023388927:execution:quetzal-immense:dfb5ebf3-53f8-46e4-903d-0a906a467ba9'

In [31]:
data = { "executionArn": executionArn }
data = json.dumps(data)
resp =  requests.post(url+'/abort',data=data,headers=headers)
resp.ok

True

In [32]:
resp.json()

{'stopDate': 1707245846.29}

In [33]:
data = { "executionArn": executionArn }
data=json.dumps(data, ensure_ascii=False).encode('utf-8')
resp =  requests.post(url+'/describe',data=data,headers=headers)
resp.ok

True

In [34]:
resp.json()

{'executionArn': 'arn:aws:states:ca-central-1:142023388927:execution:quetzal-immense:dfb5ebf3-53f8-46e4-903d-0a906a467ba9',
 'input': '{"scenario_path_S3": "test/", "launcher_arg": {"training_folder": "/tmp", "params": {"param1": "test", "param2": 3}}, "metadata": {"user_email": "test@test.com"}}',
 'inputDetails': {'__type': 'com.amazonaws.swf.base.model#CloudWatchEventsExecutionDataDetails',
  'included': True},
 'name': 'dfb5ebf3-53f8-46e4-903d-0a906a467ba9',
 'output': '{"notebook_path":"notebooks/2_model/test.ipynb","metadata":{"user_email":"test@test.com"},"scenario_path_S3":"test/","launcher_arg":{"training_folder":"/tmp","params":{"param1":"test","param2":3}}}',
 'outputDetails': {'__type': 'com.amazonaws.swf.base.model#CloudWatchEventsExecutionDataDetails',
  'included': True},
 'redriveCount': 0,
 'redriveStatus': 'NOT_REDRIVABLE',
 'redriveStatusReason': 'Execution is SUCCEEDED and cannot be redriven',
 'startDate': 1707245836.932,
 'stateMachineArn': 'arn:aws:states:ca-cent

In [35]:
resp.json()['status'] # 'ABORTED' 

'SUCCEEDED'

# S3

# S3 structure

scenario/inputs/ <br>
scenario/outputs/<br><br>

In [157]:
session = boto3.Session(
    region_name=REGION,
    aws_access_key_id=ACCESS_KEY,
    aws_secret_access_key=SECRET_KEY)

s3 = session.client('s3')

In [158]:
response = s3.list_objects_v2(Bucket=BUCKET)

In [159]:
ls=[]
for obj in response['Contents']:
    ls.append(obj["Key"])
ls

['demand/inputs/params.json',
 'orchestrator/inputs/params.json',
 'period 1/inputs/params.json',
 'period 2/inputs/params.json',
 'period 3/inputs/params.json',
 'period 4/inputs/params.json',
 'test/inputs/params.json',
 'test/inputs/population.geojson',
 'test/inputs/pt/links.geojson',
 'test/inputs/pt/nodes.geojson',
 'test/outputs/test.txt']